In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Setup
base = '../../../'

###################################
## Collapsed Parent Company File ##
###################################
# Attach to the HDF5 file where we will store the data rolled-up to the parent level
collapse_filename = base + 'Data/ReferenceUSA/parent_collapse_2015.h5'
# Setup the connection to the parent file
store = pd.HDFStore(collapse_filename)

main = store.select('main')
main=main.rename(columns = {'Company':'name'})
main=main.rename(columns = {'State':'state'})

In [ ]:
main = main[main.archive_version_year == 2015]

In [ ]:
#################
## Stripe Data ##
#################
raw_data = base + 'Data/Raw/MergeData/merchant_sample_token_name_zip_state.csv'
# Read in the ReferenceUSA data from CSV
stripe = pd.read_csv(raw_data)
stripe=stripe.rename(columns = {'legal_entity__address_state':'state'})
stripe=stripe.rename(columns = {'legal_entity__address_zip':'zip'})

In [ ]:
firm_data_name = base + 'Data/Raw/MergeData/merchant_sample.csv'
firm_data = pd.read_csv(firm_data_name)
firm_data_start = firm_data[['token', 'unified_funnel__activation_date']]
firm_data_start=firm_data_start.rename(columns = {'unified_funnel__activation_date':'start'})
firm_data_start.start = firm_data_start.start.str[0:4]

In [ ]:
stripe = stripe.merge(firm_data_start, how='left', on='token')

In [ ]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [ ]:
def clean_names(data):
    data['name'] = data.name.str.lower()
    data['name'] = data.name.str.strip()
    data['name'] = data.name.str.replace(' the ', ' ')
    data['name'] = data.name.str.replace('^the ', '')
    data['name'] = data.name.str.replace('^www\.', '')
    data['name'] = data.name.str.replace('\.com$', '')
    data['name'] = data.name.str.replace('\.co$', '')
    data['name'] = data.name.str.replace('\.net$', '')
    data['name'] = data.name.str.replace('&', 'and')
    data['name'] = data.name.str.replace("[/,_@\-\'\.\|]", '')
    data['name'] = data.name.str.replace(' co inc$', ' company incorporated')
    data['name'] = data.name.str.replace(' co. inc.$', ' company incorporated')
    data['name'] = data.name.str.replace(' co. inc. ', ' company incorporated ')
    data['name'] = data.name.str.replace(' co. inc. ', ' company incorporated ')
    data['name'] = data.name.str.replace(' co$', ' company')
    data['name'] = data.name.str.replace(' co ', ' company ')
    data['name'] = data.name.str.replace(' corp$', ' corporation')
    data['name'] = data.name.str.replace(' corp ', ' corporation ')
    data['name'] = data.name.str.replace(' inc$', ' incorporated')
    data['name'] = data.name.str.replace(' inc ', ' incorporated ')
    data['name'] = data.name.str.replace(' mfg$', ' manufacturing')
    data['name'] = data.name.str.replace(' mfg ', ' manufacturing ')
    data['name'] = data.name.str.replace(' llc$', ' limited liability company')
    data['name'] = data.name.str.replace(' llc ', ' limited liability company ')
    data['name'] = data.name.str.replace(' llp$', ' limited liability partnership')
    data['name'] = data.name.str.replace(' llp ', ' limited liability partnership ')
    data['name'] = data.name.str.replace(' fcu$', ' federal credit union')
    data['name'] = data.name.str.replace(' fcu ', ' federal credit union ')
    data['name'] = data.name.str.replace(' ltd$', ' limited')
    data['name'] = data.name.str.replace(' ltd ', ' limited ')
    data['name'] = data.name.str.replace(' svc$', ' services')
    data['name'] = data.name.str.replace(' svc ', ' services ')
    data['name'] = data.name.str.replace(' nfp$', ' not for profit')
    data['name'] = data.name.str.replace(' nfp ', ' not for profit ')
    data['name'] = data.name.str.replace(' associates$', ' associate')
    data['name'] = data.name.str.replace(' univ$', ' university')
    data['name'] = data.name.str.replace(' univ ', ' university ')
    data['name'] = data.name.str.replace(' dept$', ' department')
    data['name'] = data.name.str.replace(' dept ', ' department ')
    data['name'] = data.name.str.replace(' ctr$', ' center')
    data['name'] = data.name.str.replace(' ctr ', ' center ')
    data['name'] = data.name.str.replace(' ', '')
    return data

In [ ]:
main_clean = clean_names(main.loc[:, ['name', 'State']])

In [ ]:
clean_main = clean_names(main)

In [ ]:
MA_main = clean_main[clean_main.state == 'MA']

In [ ]:
clean_data = clean_names(stripe)

In [ ]:
combined = clean_main.merge(clean_data, how='outer', left_on=['name_2', 'state'], right_on=['name', 'state'], indicator = True)

In [ ]:
combined._merge.value_counts()

In [ ]:
combined[(combined._merge == 'right_only') & (combined.start == '2015') & (combined.state == 'MN')]['name']
a = 1

In [ ]:
combined_inc = combined[(combined.name.str.contains("incorporated") == True)]

In [ ]:
combined[(combined['name_2'].str.contains("america")==True) & (combined['name_2'].str.contains("drape")==True) ]

In [ ]:
clean_main['name_2'] = clean_main.name.str.replace(' incorporated', '')

In [ ]:
len(clean_main.name.unique())

In [ ]:
len(clean_main.name_2.unique())

In [ ]:
# Replace technology /technologies
# incorporated seems to be a problem for the match (count 2 now)
# Sub string match for side by side
# Match cehcking distance photgraphy photograhy

# Drop incorporateds if things stay unique